In [1]:
## Neural Network Exercise
## Boston Data
## Regression
## Reference: CoderzColumn Tutorial

## Steps involved in this exercise 
## Load Dataset
## Normalize Data
## Initialize Model Weights
## Activation for Hidden Layers
## Single Layer of Neural Network
## Single Forward Pass through Data to Make Predictions
## Define Loss Function
## Train Neural Network
## Make Predictions
## Evaluate Performance of Neural Network
## Train Data in Batches
## Make Predictions in Batches
## Evaluate Performance


import torch #root package
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader #dataset representation and loading

print("PyTorch Version : {}".format(torch.__version__))

PyTorch Version : 2.3.0+cu118


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device : {}".format(device))

Device : cpu


In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

#X and Y are Numpy arrays


#X, Y = datasets.load_boston(return_X_y=True)

#Ames Dataset
##from sklearn.datasets import fetch_openml
##housing = fetch_openml(name="house_prices", as_frame=True)

##Alternative way to load Boston Housing dataset
from sklearn.datasets import fetch_openml
# Fetch the Boston housing dataset
boston = fetch_openml(name='boston', version=1, parser='auto')
#X, y = boston.data, boston.target

X, Y = boston.data, boston.target

print("X Data")
print(X)
#print(X.dtype)
print(X.values)

print("Y Target")
print(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=123)


##tensorX_train = np.array(X_train.values, dtype=np.float64)
##tensorX_test = np.array(X_test.values, dtype=np.float64)
##tensorY_train = np.array(Y_train.values, dtype=np.float64)
##tensorY_test = np.array(Y_test.values, dtype=np.float64)

##change from float64 to float32
tensorX_train = np.array(X_train.values, dtype=np.float32)
tensorX_test = np.array(X_test.values, dtype=np.float32)
tensorY_train = np.array(Y_train.values, dtype=np.float32)
tensorY_test = np.array(Y_test.values, dtype=np.float32)

print(tensorX_train.dtype)


##print("X train")
##print(X_train)
##tenX_train = X_train.values
##print(tenX_train.dtype)
##The above output is Object.  This should be float or int that favours tensor

##a = np.array(tenX_train,dtype=np.float64)
##print(a.dtype)
##successfully converted Object type to float64, that is compatible with Tensor


print("X test")
print(X_test)

print("Y train")
print(Y_train)

print("Y test")
print(Y_test)


##X_train, X_test, Y_train, Y_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32), torch.tensor(Y_test, dtype=torch.float32)

##X_train, X_test, Y_train, Y_test = torch.tensor(tensorX_train, dtype = torch.float64), torch.tensor(tensorX_test, dtype=torch.float64), torch.tensor(tensorY_train, dtype=torch.float64), torch.tensor(tensorY_test, dtype=torch.float64)

X_train, X_test, Y_train, Y_test = torch.tensor(tensorX_train, dtype = torch.float32), torch.tensor(tensorX_test, dtype=torch.float32), torch.tensor(tensorY_train, dtype=torch.float32), torch.tensor(tensorY_test, dtype=torch.float32)


##X_train, X_test, Y_train, Y_test = torch.tensor(to_numpy(X_train)), torch.tensor(X_test), torch.tensor(Y_train), torch.tensor(Y_test)

##torch.tensor(X_train.to_list(),dtype=torch.float32)

##print(X_train)

##aX_train = X_train.to_numpy()
##aX_test = X_test.to_numpy()

##aY_train = Y_train.to_numpy()
##aY_test = Y_test.to_numpy()

##X_train, X_test, Y_train, Y_test = torch.tensor(aX_train,dtype=torch.float32), torch.tensor(aX_test,dtype=torch.float32), torch.tensor(aY_train, dtype=torch.float32), torch.tensor(aY_test, dtype=torch.float32)

##print(dfX.to_numpy())
##print(dfX)

##torch.tensor(dfX,dtype=torch.float32)

samples, features = X_train.shape
print(samples,features)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

X Data
        CRIM    ZN  INDUS CHAS    NOX     RM   AGE     DIS RAD    TAX  \
0    0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296.0   
1    0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242.0   
2    0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242.0   
3    0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222.0   
4    0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222.0   
..       ...   ...    ...  ...    ...    ...   ...     ...  ..    ...   
501  0.06263   0.0  11.93    0  0.573  6.593  69.1  2.4786   1  273.0   
502  0.04527   0.0  11.93    0  0.573  6.120  76.7  2.2875   1  273.0   
503  0.06076   0.0  11.93    0  0.573  6.976  91.0  2.1675   1  273.0   
504  0.10959   0.0  11.93    0  0.573  6.794  89.3  2.3889   1  273.0   
505  0.04741   0.0  11.93    0  0.573  6.030  80.8  2.5050   1  273.0   

     PTRATIO       B  LSTAT  
0       15.3  396.90   4.98  
1       17.8  396.90   9.14  
2       17.8  392.83   4.0

(torch.Size([404, 13]),
 torch.Size([102, 13]),
 torch.Size([404]),
 torch.Size([102]))

In [4]:
##Normalize the data

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

print(mean)
print(std)

X_train = (X_train - mean)/ std
X_test = (X_test - mean)/ std

print(X_train)
print(X_test)

tensor([3.5561e+00, 1.1163e+01, 1.1190e+01, 5.6931e-02, 5.5548e-01, 6.2817e+00,
        6.8975e+01, 3.7862e+00, 9.5099e+00, 4.0845e+02, 1.8464e+01, 3.5721e+02,
        1.2825e+01])
tensor([8.7882e+00, 2.2973e+01, 6.9496e+00, 2.3200e-01, 1.1639e-01, 7.0488e-01,
        2.8021e+01, 2.1380e+00, 8.6614e+00, 1.6850e+02, 2.1489e+00, 9.1677e+01,
        7.1277e+00])
tensor([[-0.4007,  3.6493, -1.2246,  ..., -1.7515,  0.3880, -1.2648],
        [ 0.7019, -0.4859,  0.9942,  ...,  0.8079,  0.3136,  0.9392],
        [-0.3888, -0.4859, -1.0274,  ..., -0.8674,  0.4329,  0.2616],
        ...,
        [-0.3647, -0.4859, -0.5483,  ...,  0.5287,  0.4329, -0.7191],
        [ 0.6407, -0.4859,  0.9942,  ...,  0.8079,  0.4329,  1.5117],
        [ 0.1138, -0.4859,  0.9942,  ...,  0.8079, -0.0274, -0.8004]])
tensor([[ 5.4140, -0.4859,  0.9942,  ...,  0.8079, -3.8680, -0.3810],
        [-0.3981, -0.4859, -0.9641,  ...,  0.0168,  0.3828, -0.8832],
        [-0.4006,  0.3847, -1.1310,  ..., -1.6585,  0.3283, -1.2

In [5]:
##Normalize weights

def InitializeWeights(layer_sizes):
    weights = []
    for i, units in enumerate(layer_sizes):
        if i==0:
            w = torch.rand(units,features, dtype=torch.float32, requires_grad=True) ## First Layer
        else:
            w = torch.rand(units,layer_sizes[i-1], dtype=torch.float32, requires_grad=True) ## All other layers

        b = torch.rand(units, dtype=torch.float32, requires_grad=True) ## Bias

        weights.append([w,b])

    return weights

torch.manual_seed(123)

weights = InitializeWeights([15,10,1])

for i, (w,b) in enumerate(weights):
    print("Layer : {}, Weights : {}, Biases : {}".format(i+1, w.shape, b.shape))

Layer : 1, Weights : torch.Size([15, 13]), Biases : torch.Size([15])
Layer : 2, Weights : torch.Size([10, 15]), Biases : torch.Size([10])
Layer : 3, Weights : torch.Size([1, 10]), Biases : torch.Size([1])


In [6]:
##Activation of Hidden Layers


def Relu(tensor):
    return torch.maximum(tensor, torch.zeros_like(tensor)) # max(0,x)

tensor = torch.tensor([-1,0,1,-2,4,-6,5])

Relu(tensor)

tensor([0, 0, 1, 0, 4, 0, 5])

In [7]:
##Single Layer of Neural Network

def LinearLayer(weights, input_data, activation=lambda x: x):
    w, b = weights
    out = torch.matmul(input_data, w.T) + b ## Multiply input by weights and add bias to it.
    return activation(out) ## Apply activation at last

rand_data = torch.rand(5, features)

out = LinearLayer(weights[0], rand_data, Relu)

print("Data Shape : {}".format(rand_data.shape))
print("Output Shape : {}".format(out.shape))

Data Shape : torch.Size([5, 13])
Output Shape : torch.Size([5, 15])


In [8]:
##Single Forward Pass through Data to Make Predictions

def ForwardPass(weights, input_data):
    layer_out = input_data

    for i in range(len(weights[:-1])):
        layer_out = LinearLayer(weights[i], layer_out, Relu) ## Hidden Layer

    preds = LinearLayer(weights[-1], layer_out) ## Final Layer

    return preds.ravel()


In [9]:
print(X_train)
##model(X_train)

preds = ForwardPass(weights, X_train)

print("Input Shape : {}, Output Shape : {}".format(X_train.shape, preds.shape))

tensor([[-0.4007,  3.6493, -1.2246,  ..., -1.7515,  0.3880, -1.2648],
        [ 0.7019, -0.4859,  0.9942,  ...,  0.8079,  0.3136,  0.9392],
        [-0.3888, -0.4859, -1.0274,  ..., -0.8674,  0.4329,  0.2616],
        ...,
        [-0.3647, -0.4859, -0.5483,  ...,  0.5287,  0.4329, -0.7191],
        [ 0.6407, -0.4859,  0.9942,  ...,  0.8079,  0.4329,  1.5117],
        [ 0.1138, -0.4859,  0.9942,  ...,  0.8079, -0.0274, -0.8004]])
Input Shape : torch.Size([404, 13]), Output Shape : torch.Size([404])


In [10]:
##Define Loss Function


def MeanSquaredErrorLoss(actual, preds):
    return torch.pow(actual - preds, 2).mean()

y1 = torch.tensor([1,2,3], dtype=torch.float32)
y2 = torch.tensor([4,5,6],dtype=torch.float32)

MeanSquaredErrorLoss(y1, y2)

tensor(9.)

In [11]:
##Train Neural Network

def TrainModel(X, Y, learning_rate, epochs):
    for i in range(1, epochs+1):
        preds = ForwardPass(weights, X) ## Make Predictions by forward pass through network

        loss = MeanSquaredErrorLoss(Y, preds) ## Calculate Loss

        loss.backward() ## Calculate Gradients

        with torch.no_grad():
            for j in range(len(weights)): ## Update Weights
                weights[j][0] -= learning_rate * weights[j][0].grad ## Update Weights
                weights[j][1] -= learning_rate * weights[j][1].grad ## Update Biases

                weights[j][0].grad = None
                weights[j][1].grad = None

        if i % 100 == 0: ## Print MSE every 100 epochs
            print("MSE : {:.2f}".format(loss))


torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 2500
learning_rate = torch.tensor(1/1e4) # 0.0001
layer_sizes = [5,10,15,1] ## Layer sizes including last layer
weights = InitializeWeights(layer_sizes) ## Initialize Weights

TrainModel(X_train, Y_train, learning_rate, epochs)

MSE : 170.44
MSE : 66.03
MSE : 31.75
MSE : 25.20
MSE : 23.12
MSE : 21.74
MSE : 20.66
MSE : 19.90
MSE : 19.23
MSE : 18.60
MSE : 18.05
MSE : 17.57
MSE : 17.13
MSE : 16.72
MSE : 16.28
MSE : 15.81
MSE : 15.39
MSE : 15.00
MSE : 14.67
MSE : 14.36
MSE : 14.09
MSE : 13.85
MSE : 13.63
MSE : 13.42
MSE : 13.24


In [12]:
## Make Predictions

train_preds = ForwardPass(weights, X_train)

train_preds[:5]

tensor([43.3787, 14.4095, 17.4788, 27.6152, 12.9184], grad_fn=<SliceBackward0>)

In [13]:
test_preds = ForwardPass(weights, X_test)

test_preds[:5]

tensor([16.6272, 27.8568, 42.6880, 15.6851, 31.4551], grad_fn=<SliceBackward0>)

In [14]:
##Evaluate performance of Neural Network

from sklearn.metrics import r2_score

print("Train R^2 Score : {:.2f}".format(r2_score(train_preds.detach().numpy(), Y_train.detach().numpy())))
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy(), Y_test.detach().numpy())))

Train R^2 Score : 0.81
Test  R^2 Score : 0.62


In [15]:
##Train Data in Batches

def UpdateWeights(weights, learning_rate):
    with torch.no_grad():
        for j in range(len(weights)): ## Update Weights
            weights[j][0] -= learning_rate * weights[j][0].grad ## Update Weights
            weights[j][1] -= learning_rate * weights[j][1].grad ## Update Biases

            weights[j][0].grad = None
            weights[j][1].grad = None

def TrainModelInBatches(X, Y, learning_rate, epochs, batch_size=32):
    for i in range(1, epochs+1):
        batches = torch.arange((X.shape[0]//batch_size)+1) ### Batch Indices

        losses = [] ## Record loss of each batch
        for batch in batches:
            if batch != batches[-1]:
                start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
            else:
                start, end = int(batch*batch_size), None

            X_batch, Y_batch = X[start:end], Y[start:end] ## Single batch of data

            preds = ForwardPass(weights, X_batch) ## Make Predictions by forward pass through network
            loss = MeanSquaredErrorLoss(Y_batch, preds) ## Calculate Loss
            losses.append(loss) ## Record Loss
            loss.backward() ## Calculate Gradients

            UpdateWeights(weights, learning_rate) ## Update Weights

        if i % 100 == 0: ## Print MSE every 100 epochs
            print("MSE : {:.2f}".format(torch.tensor(losses).mean()))



In [16]:
torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 2500
learning_rate = torch.tensor(1/1e4) # 0.0001
layer_sizes = [5,10,15,1] ## Layer sizes including last layer
weights = InitializeWeights(layer_sizes) ## Initialize Weights

TrainModelInBatches(X_train, Y_train, learning_rate, epochs)

MSE : 16.29
MSE : 13.36
MSE : 11.64
MSE : 10.51
MSE : 9.86
MSE : 9.38
MSE : 8.92
MSE : 8.49
MSE : 8.13
MSE : 7.86
MSE : 7.66
MSE : 7.50
MSE : 7.39
MSE : 7.31
MSE : 7.20
MSE : 7.03
MSE : 6.87
MSE : 6.76
MSE : 6.65
MSE : 6.56
MSE : 6.46
MSE : 6.38
MSE : 6.27
MSE : 6.17
MSE : 6.08


In [17]:
##Make Predictions in Batches

def MakePredictions(input_data, batch_size=32):
    batches = torch.arange((input_data.shape[0]//batch_size)+1) ### Batch Indices

    with torch.no_grad(): ## Disables automatic gradients calculations
        preds = []
        for batch in batches:
            if batch != batches[-1]:
                start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
            else:
                start, end = int(batch*batch_size), None

            X_batch = input_data[start:end]

            preds.append(ForwardPass(weights, X_batch))

    return preds

In [18]:
test_preds = MakePredictions(X_test)

test_preds = torch.cat(test_preds)

train_preds = MakePredictions(X_train)

train_preds = torch.cat(train_preds)

In [19]:
from sklearn.metrics import r2_score

print("Train R^2 Score : {:.2f}".format(r2_score(train_preds.detach().numpy(), Y_train.detach().numpy())))
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy(), Y_test.detach().numpy())))

Train R^2 Score : 0.92
Test  R^2 Score : 0.70
